In [1]:
import numpy as np

def one_hot(y):
    n_classes = np.unique(y)
    one_hot = np.zeros((len(y), len(n_classes)))
    for i, c in enumerate(y):
        one_hot[i, n_classes == c] = 1
    return one_hot

# Assuming you have four classes represented by numbers 0 to 3
y = np.array([0, 1, 2, 3, 0, 2, 1, 3, 0])

# Convert classes to one-hot encoding
one_hot_y = one_hot(y)

print(one_hot_y)


[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]


In [2]:

import numpy as np

def softmax(z):
    exps = np.exp(z)
    sum_exps = np.sum(exps)
    return exps / sum_exps

z = np.array([1.0, 2.0, 3.0])
probabilities = softmax(z)

print(probabilities)


[0.09003057 0.24472847 0.66524096]


## Logestic Regression Breakdown

In [1]:
import numpy as np


class LogisticRegression:
    def __init__(self, lr=0.00001, n_iter=1000):
        self.lr = lr
        self.n_iter = n_iter
        self.w = None

    @staticmethod
    def one_hot(y):
        n_classes = np.unique(y)
        one_hot = np.zeros((len(y), len(n_classes)))
        for i, c in enumerate(y):
            one_hot[i, n_classes == c] = 1
        return one_hot

    def probabilities(self, X):
        z = np.dot(X, self.w.T)
        return np.exp(z) / np.sum(np.exp(z), axis=1, keepdims=True)

    def predict(self, X):
        X = np.insert(X, 0, 1, axis=1)
        return np.argmax(self.probabilities(X), axis=1)

    def accuracy(self, X, y):
        return np.mean(self.predict(X) == y)

    def fit(self, X, y):
        X = np.insert(X, 0, 1, axis=1) # (samples,dim) -> (samples, dim + 1)
        self.w = np.zeros((len(np.unique(y)), X.shape[1])) # (4, 301)
        y = self.one_hot(y)

        for _ in range(self.n_iter):
            predictions = self.probabilities(X)
            error = predictions - y
            gradient = np.dot(error.T, X)
            self.w -= self.lr * gradient
            

In [3]:
# Generating random data
np.random.seed(0) # For consistent results
X = np.random.rand(100, 300) # 100 samples, 300 features each
y = np.random.choice([0, 1, 2, 3], 100) # 100 targets belonging to one of four classes


# Creating an instance of the Logistic Regression class
model = LogisticRegression(lr=0.00001, n_iter=1)


In [4]:
# Adding bias to the feature matrix
X = np.insert(X, 0, 1, axis=1)
print(f'Shape of X after adding bias: {X.shape}')  # (100, 301)

# Initializing weights
model.w = np.zeros((len(np.unique(y)), X.shape[1]))
print(f'Initial shape of weights: {model.w.shape}')  # (4, 301)

# One-hot encoding the target variable
y = model.one_hot(y)
print(f'Shape of y after one-hot encoding: {y.shape}')  # (100, 4)

for _ in range(model.n_iter):
    # Compute probabilities
    predictions = model.probabilities(X)
    print(f'Shape of predictions: {predictions.shape}')  # (100, 4)

    # Compute error
    error = predictions - y
    print(f'Shape of error: {error.shape}')  # (100, 4)

    # Compute gradient
    gradient = np.dot(error.T, X)
    print(f'Shape of gradient: {gradient.shape}')  # (4, 301)

    # Update weights
    model.w -= model.lr * gradient
    print(f'Shape of weights after update: {model.w.shape}')  # (4, 301)


Shape of X after adding bias: (100, 301)
Initial shape of weights: (4, 301)
Shape of y after one-hot encoding: (100, 4)
Shape of predictions: (100, 4)
Shape of error: (100, 4)
Shape of gradient: (4, 301)
Shape of weights after update: (4, 301)
